<a href="https://colab.research.google.com/github/AinzOwl/mysticai-colab/blob/main/Mystic_Civitai_SD1_5_%5Bhf_method%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mystic.Ai SD 1.5 civitai model deployment script**

Hello, I'm [Ainzoal](https://ainz.sbs) you can find me on mystic.ai discord server, I'm back again with a new colab, this one is for deploying models from civitai to mistic.ai for an api inference, it is by no means the best way to do it but it works so yeah.

the way this one work is by converting the model to diffusers then upload it to huggingface after that import it to mystic from huggingface like we did in 1.5 colab *(I told you its not best way)*

Well enjoy and for any question or help yk where to find me.

For SDXL [click here](https://github.com/AinzOwl/mysticai-colab/blob/main/mystic_ai_sdxl.ipynb)

For normal V1.5 [click here](https://github.com/AinzOwl/mysticai-colab/blob/main/mystic_ai_sd1_5.ipynb)

Notebooks Repo: [click here](https://github.com/AinzOwl/mysticai-colab)

In [ ]:
#@title Install Packages
!pip install pipeline-ai torch xformers
!pip install -q transformers==4.31.0
!pip install -q accelerate==0.21.0
!pip install -q diffusers==0.20.0
!pip install -q huggingface_hub==0.16.4
!pip install -q omegaconf==2.3.0

# **Download Model**

First thing we need to download the model from civitai to do that right click the blue download button in https://civitai.com/ for the model you want, copy link and paste it in this one then click **enter**

it will also ask you for extension it can be safetensors or like ckpt depending on your model, just type it there for example if model is a safetensor type `safetensors` and no **.** is needed

In [ ]:
#@title model download
path = input("model path please: ")
modelext = input("model extension pls: ")
!curl -Lo model.{modelext} {path}
# this next line will download the converter, dw its from huggingface themselves
!wget https://raw.githubusercontent.com/huggingface/diffusers/v0.20.0/scripts/convert_original_stable_diffusion_to_diffusers.py

#**Convert**

Thie cell is the one responsible for converting normal model into a diffusers ready one, just run it and wait a bit.

In [ ]:
!python convert_original_stable_diffusion_to_diffusers.py --checkpoint_path model.{modelext} --dump_path model/ --from_safetensors

#**Upload to huggingface**

Those next cells will upload to huggingface, first you need tu supply your api key must be write since it will upload you can get it from [Here](https://huggingface.co/settings/tokens)

Next you will have to create a new public model manually from [Here](https://huggingface.co/new), its best if you name it the same name of the civitai model so other people can use it, but its all up to you.

Next run the second cell it will ask you for the id of the model you just created u can get it from the model page by clicking the copy icon

In [ ]:
#@title model download
hfapikey = input("model path please: ")
!huggingface-cli login --token {hfapikey}

# **Upload**

Note before running this one make sure you have created a new model in huggingface using this [link](https://huggingface.co/new)

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

modelName = input("Enter huggingface model id: ")
api.upload_folder(
    folder_path="model",
    repo_id=modelName,
    repo_type="model",
)

# **Mystic**

Now we upload it to mystic.ai this is almost same as sd1.5 notebook so i'll be copy pasting most of notes 😂

# **setup api key**

When you run this it will ask you for your api key if you don't know where to find it it's [Here](https://mystic.ai/api-tokens) just type it and hit enter

In [ ]:
#@title Setup Mystic.Ai Api Key

api_key = input('Enter mystic.ai api key: ')
!pipeline cluster login catalyst-api {api_key} -u https://www.mystic.ai -a

# **import packages**
Yeah just run this one and that's it, nothing to change

In [ ]:
#@title Import packages
from pathlib import Path
from typing import List

import torch
from diffusers import StableDiffusionPipeline, AutoencoderKL

from pipeline import Pipeline, Variable, entity, pipe
from pipeline.cloud import compute_requirements, environments, pipelines
from pipeline.objects import Directory, File
from pipeline.objects.graph import InputField, InputSchema

# **inputs**

this is the configuration for the model inputs, you don't really need to change anything here unless you want to restrict some stuff or change some default values

to change default values it's easy just search for name of value you want to change and change its default, for example you want to set default height to 1024 then go to height and change 512 with 1024

```There's a bug that makes empty default inputs required, I just figured out you can bypass it by putting a space in default so yeah thats why there's a space for negative prompt if you are wondering```

In [ ]:
#@title Setup inputs

class ModelKwargs(InputSchema):
    height: int | None = InputField(
        title="Height",
        default=512,
        ge=64,
        le=1024,
        multiple_of=8,
    )

    width: int | None = InputField(
        title="Width",
        default=512,
        ge=64,
        le=1024,
        multiple_of=8,
    )

    num_inference_steps: int | None = InputField(
        title="Number of inference steps",
        default=25,
    )

    num_images_per_prompt: int | None = InputField(
        default=1, title="Number of images", ge=1, le=4
    )

    negative_prompt: str  | None = InputField(
        default=' ',
        title="Negative Prompt",
    )

    guidance_scale: float | None = InputField(
        default=7.5,
        title="Guidance scale",
        ge=0.0,
        le=20.0,
        # multiple_of=0.1,
    )

    allow_nsfw: bool | None = InputField(
        default=True,
        title="Allow NSFW",
    )

# **Setting up the model**

This one require a small input from you, just open it and change model_path with your model id from hugggingface (the one we just made, also if you go check it you'll find a bunch of files there ^^), to easily find it in code look for ``ainz/hineseLandscapeArt``



In [ ]:
#@title Setup Model
@entity
class StableDiffusionModel:
    def __init__(self):
        ...

    @pipe(on_startup=True, run_once=True)
    def load(self):

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model_path = "ainz/hineseLandscapeArt"
        self.pipe = StableDiffusionPipeline.from_pretrained(
            model_path,
            torch_dtype=torch.float16
        )
        self.pipe = self.pipe.to(device)

        def disabled_safety_checker(images, clip_input):
            if len(images.shape) == 4:
                num_images = images.shape[0]
                return images, [False] * num_images
            else:
                return images, False

        self.no_filter = disabled_safety_checker
        self.nsfw_filter = self.pipe.safety_checker

    @pipe
    def predict(self, prompt: str, kwargs: ModelKwargs) -> List[File]:
        defaults = kwargs.to_dict()
        del defaults["allow_nsfw"]

        if kwargs.allow_nsfw:
            self.pipe.safety_checker = self.no_filter

        images = self.pipe(prompt, **defaults).images

        output_images = []
        for i, image in enumerate(images):
            path = Path(f"/tmp/sd/image-{i}.jpg")
            path.parent.mkdir(parents=True, exist_ok=True)
            image.save(str(path))
            output_images.append(File(path=path, allow_out_of_context_creation=True))

        return output_images

# **Environment**

Env is the one holding the packages that your model uses, the ones selected are the ones needed to get this diffusers model to run and same env can be used for different models so you kinda really just need to run this one ones and you're good

In [ ]:
#@title setup mystic environment

with Pipeline() as builder:
    prompt = Variable(
        str,
        title="Prompt"
    )
    kwargs = Variable(
        ModelKwargs,
        title="Model kwargs",
    )

    model = StableDiffusionModel()

    model.load()

    output = model.predict(prompt, kwargs)

    builder.output(output)

my_pl = builder.get_pipeline()
try:
    environments.create_environment(
        "stable_diffusion_1_5",
        python_requirements=[
            "torch==2.0.1",
            "transformers==4.30.2",
            "diffusers==0.19.3",
            "accelerate==0.21.0",
            "xformers==0.0.21",
        ],
    )
except Exception:
    pass

# **Pushing the model**

pushing is easy you just replace `hineseLandscapeArt` with what you named the model and thats it, if you used a different name for environement you can change it.

For vram it depends on model how much it need but dont overkill it just try to find the balance.

For accelerators check this page [Mystic Ai Docs](https://docs.mystic.ai/docs/gpus-and-accelerators) you'll find the id of each and pick one you want depending on price and vram

In [ ]:
#@title Push the model (open this to change required gpu type)
remote_pipeline = pipelines.upload_pipeline(
    my_pl,
    "hineseLandscapeArt:latest",
    environment_id_or_name="stable_diffusion_1_5_civit",
    required_gpu_vram_mb=20_000,
    accelerators=[
        compute_requirements.Accelerator.nvidia_a100,
    ],
)

print(remote_pipeline.id)